In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime

from tqdm.notebook import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/ecg-retrieval-integrated11'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
!pip install biosppy
!pip install pyentrp
!pip install pyhrv
!pip install PyWavelets
#!pip install heartpy
#!git clone https://github.com/physiozoo/mhrv.git

     |████████████████████████████████| 76 kB 329 kB/s 
  Created wheel for pyentrp: filename=pyentrp-0.6.0-py2.py3-none-any.whl size=5040 sha256=b929e5f99e13549d669f50d99daa0513edc70e313e0877cb330adfb141b3eb6d
  Stored in directory: /root/.cache/pip/wheels/74/87/62/e61a7ae7fcbc8fd2864c663c1bd3edd63ddd574bbbdb6cccb2
Successfully built pyentrp
     |████████████████████████████████| 3.2 MB 402 kB/s 
     |████████████████████████████████| 227 kB 8.2 MB/s 
  Created wheel for spectrum: filename=spectrum-0.7.6-cp37-cp37m-linux_x86_64.whl size=235359 sha256=e8fa1c4b99bfa925eb7cacd6aa6410ff460b45247a61b40c7b695c2f047a0f39
  Stored in directory: /root/.cache/pip/wheels/ef/78/be/cbfb44e11a46428212320583883c35e907785f660dc35ee1a6
Successfully built spectrum


In [2]:
"""import numpy as np
a = np.random.rand(5000)
b = np.random.rand(5000)
a = (a - np.mean(a)) / (np.std(a) * len(a))
b = (b - np.mean(b)) / (np.std(b))
np.correlate(a, b)"""

'import numpy as np\na = np.random.rand(5000)\nb = np.random.rand(5000)\na = (a - np.mean(a)) / (np.std(a) * len(a))\nb = (b - np.mean(b)) / (np.std(b))\nnp.correlate(a, b)'

In [3]:
import biosppy
import base64
import pyentrp.entropy as ent
import pyhrv
import pyhrv.time_domain as td
import pyhrv.frequency_domain as fd
import matplotlib.pyplot as plt

from scipy.stats import skew, kurtosis

import re

fs = 500
    
def getLeadFeatures(lead_data):
        features = []
        signal = biosppy.ecg.ecg(lead_data, show=False, sampling_rate=fs)
        rpeaks = signal['rpeaks']
        if rpeaks.shape[0] != 0:
            
            rr_intervals = np.diff(rpeaks)
            drr = np.diff(rr_intervals)
            r_density = (rr_intervals.shape[0] + 1) / lead_data.shape[0] * fs
            pnn50 = drr[drr >= fs * 0.05].shape[0] / rr_intervals.shape[0]
            rmssd = np.sqrt(np.mean(drr * drr))
            sdnn = td.sdnn(rr_intervals)['sdnn']
            rmssd = td.rmssd(rr_intervals)['rmssd']
            #fft = fd.welch_psd(rr_intervals, show=False)['fft_ratio']
            #fft = 1
            
            samp_entrp = ent.sample_entropy(rr_intervals, 2, 0.2 * np.std(rr_intervals))
            samp_entrp[np.isnan(samp_entrp)] = -2
            samp_entrp[np.isinf(samp_entrp)] = -1
            
            features.extend([rr_intervals.min(),
                             rr_intervals.max(),
                             rr_intervals.mean(),
                             rr_intervals.std(),
                             skew(rr_intervals),
                             kurtosis(rr_intervals),
                             r_density,
                             samp_entrp[0], samp_entrp[1],
                             sdnn,
                             rmssd,
                             pnn50,
                             skew(lead_data),
                             kurtosis(lead_data)
                            ])
        else:
            features.extend([np.nan] * 16)
        
        heart_rate = signal['heart_rate']
        if heart_rate.shape[0] != 0:
            features.extend([heart_rate.min(),
                                 heart_rate.max(),
                                 heart_rate.mean(),
                                 heart_rate.std(),
                                 skew(heart_rate),
                                 kurtosis(heart_rate)])
        else:
            features.extend([np.nan] * 6)
        return features
    
def getFeatures(index_data, index_feature):
    
        age_s = index_feature.find('Age')
        if age_s == -1: age_s = index_feature.find('age')
        if age_s == -1: 
            age = 0
        else:
            age_s = re.search(r"\d+", index_feature[age_s:]).start() + age_s
            age_e = re.search(r"\D+", index_feature[age_s:]).start() + age_s
            age = int(index_feature[age_s:age_e])

        sex = index_feature.find('Female')
        if sex == -1: sex = index_feature.find('female')
        if sex == -1: sex = index_feature.find('Sex: F')    
        if sex != -1: sex = 0
        else: sex = 1
            
        #print(sex)
        #print(age)

        index_data = base64.b64decode(index_data)
        index_data = np.frombuffer(index_data, dtype=np.float16).reshape(-1,12).transpose().astype(np.float32)
        features = [sex,age]
        
        for lead_data in index_data:
            features.extend(getLeadFeatures(lead_data))
            
        return features

In [4]:
for dirname, _, filenames in os.walk('/kaggle/input/prepare-integrated-data2'):
    for filename in tqdm(filenames):
        with open(os.path.join(dirname, filename)) as f:
            try:
                file_name, index_data, index_label, index_feature = f.readline().strip().split('\t')  
                try:
                    features = np.array(getFeatures(index_data, index_feature))
                except:
                    features = np.array([0] * 242)
                features = base64.b64encode(features.astype(np.float16)).decode("utf-8")
                out_f = open(filename,'w+')
                out_f.write('\t'.join([file_name, index_data, index_label, index_feature, features]))
            except:
                pass
            #print("index_feature ",index_feature)
            #print(index_feature)

In [5]:
"""for dirname, _, filenames in os.walk('/kaggle/input/prepare-integrated-data2'):
    for filename in tqdm(filenames):
        with open(os.path.join(dirname, filename)) as f:
            if filename.find("ecg_data") == -1: print(filename)"""

'for dirname, _, filenames in os.walk(\'/kaggle/input/prepare-integrated-data2\'):\n    for filename in tqdm(filenames):\n        with open(os.path.join(dirname, filename)) as f:\n            if filename.find("ecg_data") == -1: print(filename)'

In [6]:
!cp /kaggle/input/prepare-integrated-data2/ori_round1_label_to_123_lalel ori_round1_label_to_123_lalel
!cp /kaggle/input/prepare-integrated-data2/ori_round2_label_to_123_lalel ori_round2_label_to_123_lalel
!cp /kaggle/input/prepare-integrated-data2/ori_edar_label_to_123_lalel ori_edar_label_to_123_lalel